<h1 style='color:white; background:#0A0502; border:0'><center>RANZCR: Xception TPU</center></h1>

![](https://storage.googleapis.com/kaggle-competitions/kaggle/23870/logos/header.png?t=2020-12-01-04-28-05)

Serious complications can occur as a result of malpositioned lines and tubes in patients. Doctors and nurses frequently use checklists for placement of lifesaving equipment to ensure they follow protocol in managing patients. Yet, these steps can be time consuming and are still prone to human error, especially in stressful situations when hospitals are at capacity.

Hospital patients can have catheters and lines inserted during the course of their admission and serious complications can arise if they are positioned incorrectly. Nasogastric tube malpositioning into the airways has been reported in up to 3% of cases, with up to 40% of these cases demonstrating complications [1-3]. Airway tube malposition in adult patients intubated outside the operating room is seen in up to 25% of cases [4,5]. The likelihood of complication is directly related to both the experience level and specialty of the proceduralist. Early recognition of malpositioned tubes is the key to preventing risky complications (even death), even more so now that millions of COVID-19 patients are in need of these tubes and lines.

<a id="start"></a>

The gold standard for the confirmation of line and tube positions are chest radiographs. However, a physician or radiologist must manually check these chest x-rays to verify that the lines and tubes are in the optimal position. Not only does this leave room for human error, but delays are also common as radiologists can be busy reporting other scans. Deep learning algorithms may be able to automatically detect malpositioned catheters and lines. Once alerted, clinicians can reposition or remove them to avoid life-threatening complications.

<h2 style='color:#0A0502; background:white; border:2px solid #0A0502'><center>Table of contents:</center></h2>

* [**Fast look at the data**](#1)
* [**Preparation for modeling**](#2)
* [**Training**](#3)
* [**Visualization of CNN intermediate activations**](#4)
* [**Prediction**](#5)


## Prediction version of this notebook: [RANZCR: Xception TPU Prediction](https://www.kaggle.com/maksymshkliarevskyi/ranzcr-xception-tpu-prediction)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.applications import Xception
from tensorflow.keras.optimizers import Adam
from kaggle_datasets import KaggleDatasets
import tensorflow_addons as tfa

# ignoring warnings
import warnings
warnings.simplefilter("ignore")

import os, cv2

In [ ]:
# TPU or GPU detection
# Detect hardware, return appropriate distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print(f'Running on TPU {tpu.master()}')
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy()

AUTO = tf.data.experimental.AUTOTUNE
REPLICAS = strategy.num_replicas_in_sync
print(f'REPLICAS: {REPLICAS}')

<h3 style='color:white; background:#0A0502; border:0'><center>Work directory</center></h3>

In [ ]:
WORK_DIR = '../input/ranzcr-clip-catheter-line-classification'
os.listdir(WORK_DIR)

<a id="1"></a>
<h1 style='color:white; background:#0A0502; border:0'><center>Fast look at the data</center></h1>

[**Back to the table of contents**](#start)

In [ ]:
print('Train images: %d' %len(os.listdir(os.path.join(WORK_DIR, "train"))))

In [ ]:
# Data
GCS_DS_PATH = KaggleDatasets().get_gcs_path('ranzcr-clip-catheter-line-classification')

train = pd.read_csv(os.path.join(WORK_DIR, "train.csv"))
train_images = GCS_DS_PATH + "/train/" + train['StudyInstanceUID'] + '.jpg'

ss = pd.read_csv(os.path.join(WORK_DIR, 'sample_submission.csv'))
test_images = GCS_DS_PATH + "/test/" + ss['StudyInstanceUID'] + '.jpg'

label_cols = ss.columns[1:]
labels = train[label_cols].values

train_annot = pd.read_csv(os.path.join(WORK_DIR, "train_annotations.csv"))

print('Labels:\n', '*'*20, '\n', label_cols.values)
print('*'*50)
train.head()

In [ ]:
sns.set_style("whitegrid")
fig = plt.figure(figsize = (15, 12), dpi = 300)
plt.suptitle('Labels count', fontfamily = 'serif', size = 15)

for ind, i in enumerate(label_cols):
    fig.add_subplot(4, 3, ind + 1)

    sns.countplot(train[i], edgecolor = 'black',
                  palette = reversed(sns.color_palette('viridis', 2)))
    
    plt.xlabel('')
    plt.ylabel('')
    plt.xticks(fontfamily = 'serif', size = 10)
    plt.yticks(fontfamily = 'serif', size = 10)
    plt.title(i, fontfamily = 'serif', size = 10)
plt.show()

<h3 style='color:white; background:#0A0502; border:0'><center>Some images</center></h3>

In [ ]:
sample = train.sample(9)
plt.figure(figsize = (10, 7), dpi = 300)
for ind, image_id in enumerate(sample.StudyInstanceUID):
    plt.subplot(3, 3, ind + 1)
    image = image_id + '.jpg'
    img = cv2.imread(os.path.join(WORK_DIR, "train", image))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img)
    plt.title('Shape: {}'.format(img.shape[:2]))
    plt.axis("off")
plt.tight_layout()
plt.show()

<a id="2"></a>
<h1 style='color:white; background:#0A0502; border:0'><center>Preparation for modeling</center></h1>

[**Back to the table of contents**](#start)

In [ ]:
# Main parameters
BATCH_SIZE = 8 * REPLICAS
STEPS_PER_EPOCH = len(train) * 0.85 / BATCH_SIZE
VALIDATION_STEPS = len(train) * 0.15 / BATCH_SIZE
EPOCHS = 30
TARGET_SIZE = 750

<h2 style='color:white; background:#0A0502; border:0'><center>Functions</center></h2>

I'm thankful to **xhlulu** for useful functions and methods of working with TPU ([RANZCR: EfficientNet B3 GPU Starter](https://www.kaggle.com/xhlulu/ranzcr-efficientnet-b3-gpu-starter))

In [ ]:
def build_decoder(with_labels = True,
                  target_size = (TARGET_SIZE, TARGET_SIZE), 
                  ext = 'jpg'):
    def decode(path):
        file_bytes = tf.io.read_file(path)
        if ext == 'png':
            img = tf.image.decode_png(file_bytes, channels = 3)
        elif ext in ['jpg', 'jpeg']:
            img = tf.image.decode_jpeg(file_bytes, channels = 3)
        else:
            raise ValueError("Image extension not supported")

        img = tf.cast(img, tf.float32) / 255.0
        img = tf.image.resize(img, target_size)

        return img
    
    def decode_with_labels(path, label):
        return decode(path), label
    
    return decode_with_labels if with_labels else decode


def build_augmenter(with_labels = True):
    def augment(img):
        img = tf.image.random_flip_left_right(img)
        img = tf.image.random_flip_up_down(img)
        img = tf.image.adjust_brightness(img, 0.1)
        
#         rotate = tf.random.uniform([], 0, 1.0, dtype = tf.float32)      
#         if rotate > .75:
#             img = tf.image.rot90(img, k = 3)
#         elif rotate > .5:
#             img = tf.image.rot90(img, k = 2)
#         elif rotate > .25:
#             img = tf.image.rot90(img, k = 1)
        
#         saturation = tf.random.uniform([], 0, 1.0, dtype = tf.float32)
#         if saturation >= .5:
#             img = tf.image.random_saturation(img, lower = 0.9, upper = 1.1)
        
#         contrast = tf.random.uniform([], 0, 1.0, dtype = tf.float32)
#         if contrast >= .5:
#             img = tf.image.random_contrast(img, lower = 0.9, upper = 1.1)
        
#         brightness = tf.random.uniform([], 0, 1.0, dtype = tf.float32)
#         if brightness >= .5:
#             img = tf.image.random_brightness(img, max_delta = 0.1)
        
        return img
    
    def augment_with_labels(img, label):
        return augment(img), label
    
    return augment_with_labels if with_labels else augment


def build_dataset(paths, labels = None, bsize = 32, cache = True,
                  decode_fn = None, augment_fn = None,
                  augment = True, repeat = True, shuffle = 1024, 
                  cache_dir = ""):
    if cache_dir != "" and cache is True:
        os.makedirs(cache_dir, exist_ok=True)
    
    if decode_fn is None:
        decode_fn = build_decoder(labels is not None)
    
    if augment_fn is None:
        augment_fn = build_augmenter(labels is not None)
    
    AUTO = tf.data.experimental.AUTOTUNE
    slices = paths if labels is None else (paths, labels)
    
    dset = tf.data.Dataset.from_tensor_slices(slices)
    dset = dset.map(decode_fn, num_parallel_calls = AUTO)
    dset = dset.cache(cache_dir) if cache else dset
    dset = dset.map(augment_fn, num_parallel_calls = AUTO) if augment else dset
    dset = dset.repeat() if repeat else dset
    dset = dset.shuffle(shuffle) if shuffle else dset
    dset = dset.batch(bsize).prefetch(AUTO)
    
    return dset

In [ ]:
# Train test split
(train_img, valid_img, 
 train_labels, valid_labels) = train_test_split(train_images, labels, 
                                                train_size = 0.85, 
                                                random_state = 0)

In [ ]:
# Tensorflow datasets
train_df = build_dataset(
    train_img, tf.cast(train_labels, tf.float32), bsize = BATCH_SIZE, 
    cache = True)

valid_df = build_dataset(
    valid_img, tf.cast(valid_labels, tf.float32), bsize = BATCH_SIZE, 
    repeat = False, shuffle = False, augment = False, 
    cache = True)

test_df = build_dataset(
    test_images, bsize = BATCH_SIZE, repeat = False, 
    shuffle = False, augment = False, cache = False)

In [ ]:
train_df

In [ ]:
def create_model():
    conv_base = Xception(include_top = False, weights = 'imagenet',
                         input_shape = (TARGET_SIZE, TARGET_SIZE, 3))
    model = conv_base.output
    model = layers.GlobalAveragePooling2D()(model)
    model = layers.Dropout(0.3)(model)
    model = layers.Dense(11, activation = "sigmoid")(model)
    model = models.Model(conv_base.input, model)

    model.compile(optimizer = Adam(lr = 0.001),
                  loss = tfa.losses.SigmoidFocalCrossEntropy(alpha = 0.5, gamma = 2),
                  metrics = [tf.keras.metrics.AUC(multi_label = True)])
    return model

In [ ]:
with strategy.scope():
    model = create_model()
    
model.summary()

In [ ]:
print('Our Xception CNN has %d layers' %len(model.layers))

<a id="3"></a>
<h1 style='color:white; background:#0A0502; border:0'><center>Training</center></h1>

[**Back to the table of contents**](#start)

In [ ]:
model_save = ModelCheckpoint('./Xcep_750_best_weights_TPU.h5', 
                             save_best_only = True, 
                             save_weights_only = True,
                             monitor = 'val_loss', 
                             mode = 'min', verbose = 1)
early_stop = EarlyStopping(monitor = 'val_loss', min_delta = 0.001, 
                           patience = 5, mode = 'min', verbose = 1,
                           restore_best_weights = True)
reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.5, 
                              patience = 2, min_delta = 0.0001, 
                              mode = 'min', verbose = 1)


history = model.fit(
    train_df,
    epochs = EPOCHS,
    steps_per_epoch = STEPS_PER_EPOCH,
    validation_data = valid_df,
    validation_steps = VALIDATION_STEPS,
    callbacks = [model_save, early_stop, reduce_lr]
)

In [ ]:
auc = history.history['auc']
val_auc = history.history['val_auc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(auc) + 1)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (15, 5))
sns.set_style("white")
plt.suptitle('Train history', size = 15)

ax1.plot(epochs, auc, "bo", label = "Training auc")
ax1.plot(epochs, val_auc, "b", label = "Validation auc")
ax1.set_title("Training and validation auc")
ax1.legend()

ax2.plot(epochs, loss, "bo", label = "Training loss", color = 'red')
ax2.plot(epochs, val_loss, "b", label = "Validation loss", color = 'red')
ax2.set_title("Training and validation loss")
ax2.legend()

plt.show()

In [ ]:
model.save('./Xception_750_TPU.h5')

<a id="4"></a>
<h1 style='color:white; background:#0A0502; border:0'><center>Visualization of CNN intermediate activations</center></h1>

[**Back to the table of contents**](#start)

In [ ]:
def activation_layer_vis(img, activation_layer = 0, layers = 10):
    layer_outputs = [layer.output for layer in model.layers[:layers]]
    activation_model = models.Model(inputs = model.input, outputs = layer_outputs)
    activations = activation_model.predict(img)
    
    rows = int(activations[activation_layer].shape[3] / 3)
    cols = int(activations[activation_layer].shape[3] / rows)
    fig, axes = plt.subplots(rows, cols, figsize = (15, 15 * cols))
    axes = axes.flatten()
    
    for i, ax in zip(range(activations[activation_layer].shape[3]), axes):
        ax.matshow(activations[activation_layer][0, :, :, i], cmap = 'viridis')
        ax.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
img_tensor = build_dataset(
    pd.Series(train_img[0]), bsize = 1,repeat = False, 
    shuffle = False, augment = False, cache = False)

<h3 style='color:white; background:#0A0502; border:0'><center>Visualization of the first layer</center></h3>

In [ ]:
activation_layer_vis(img_tensor)

In [ ]:
def all_activations_vis(img, layers = 10):
    layer_outputs = [layer.output for layer in model.layers[:layers]]
    activation_model = models.Model(inputs = model.input, outputs = layer_outputs)
    activations = activation_model.predict(img)
    
    layer_names = []
    for layer in model.layers[:layers]: 
        layer_names.append(layer.name) 

    images_per_row = 3
    for layer_name, layer_activation in zip(layer_names, activations): 
        n_features = layer_activation.shape[-1] 

        size = layer_activation.shape[1] 

        n_cols = n_features // images_per_row 
        display_grid = np.zeros((size * n_cols, images_per_row * size)) 

        for col in range(n_cols): 
            for row in range(images_per_row): 
                channel_image = layer_activation[0, :, :, col * images_per_row + row] 
                channel_image -= channel_image.mean() 
                channel_image /= channel_image.std() 
                channel_image *= 64 
                channel_image += 128 
                channel_image = np.clip(channel_image, 0, 255).astype('uint8') 
                display_grid[col * size : (col + 1) * size, 
                             row * size : (row + 1) * size] = channel_image 
        scale = 1. / size 
        plt.figure(figsize=(scale * 5 * display_grid.shape[1], 
                            scale * 5 * display_grid.shape[0])) 
        plt.title(layer_name) 
        plt.grid(False)
        plt.axis('off')
        plt.imshow(display_grid, aspect = 'auto', cmap = 'viridis')

<h3 style='color:white; background:#0A0502; border:0'><center>Visualization of the first 3 layers</center></h3>

In [ ]:
all_activations_vis(img_tensor, 3)

<a id="5"></a>
<h1 style='color:white; background:#0A0502; border:0'><center>Prediction</center></h1>

[**Back to the table of contents**](#start)

In [ ]:
ss[label_cols] = model.predict(test_df)
ss.to_csv('submission.csv', index = False)

In [ ]:
ss.head()

## Prediction version of this notebook: [RANZCR: Xception TPU Prediction](https://www.kaggle.com/maksymshkliarevskyi/ranzcr-xception-tpu-prediction)

<h1 style='color:white; background:#0A0502; border:0'><center>WORK IN PROGRESS...</center></h1>